In [14]:
import sys


path = "/var/genie-in-the-box/src/lib"
sys.path.append( path )

import util as du
du.add_to_path( path )

Path [/var/genie-in-the-box/src/lib] already in sys.path


In [8]:
import time

class Stopwatch:

    def __enter__( self ):

        self.start = time.time()

        return self

    def __exit__( self, *args ):

        self.end      = time.time()
        self.interval = int( ( self.end - self.start ) * 1000 )

        print( f'Elapsed time: {self.interval} milliseconds' )

with Stopwatch() as sw:

    time.sleep( 1 )

Elapsed time: 1001 milliseconds


In [37]:
import os
import glob
import json
from datetime import datetime

import openai
import numpy as np

class SolutionSnapshot:

    def __init__( self, problem, solution_summary="", code="", programming_language="python", language_version="3.10",
                  solution_embedding=[ ], code_embedding=[ ], solution_directory="/src/conf/long-term-memory/solutions/"
        ):

        self.created_date         = self._get_timestamp()
        self.updated_date         = self.created_date
        self.problem              = problem
        self.solution_summary     = solution_summary
        self.code                 = code
        self.programming_language = programming_language
        self.language_version     = language_version
        self.solution_directory   = solution_directory
        self.solution_file        = None

        self.problem_embedding    = self._generate_embedding( problem )
        self.solution_embedding   = solution_embedding
        self.code_embedding       = code_embedding

    def _get_timestamp( self ):

        now = datetime.now()
        return now.strftime( "%Y-%m-%d @ %H-%M-%S" )

    def set_solution_summary( self, solution_summary ):

        self.solution_summary   = solution_summary
        self.solution_embedding = self._generate_embedding( solution_summary )
        self.updated_date       = self._get_timestamp()

    def set_code( self, code ):

        self.code           = code
        self.code_embedding = self._generate_embedding( code )
        self.updated_date   = self._get_timestamp()

    def _generate_embedding( self, text ):

        with Stopwatch() as sw:
            response = openai.Embedding.create(
                input=text,
                model="text-embedding-ada-002"
            )
        return response[ "data" ][ 0 ][ "embedding" ]

    def get_problem_similarity( self, other_snapshot ):
        if not self.problem_embedding or not other_snapshot.problem_embedding:
            raise ValueError( "Both snapshots must have an embedding to compare." )
        return np.dot( self.problem_embedding, other_snapshot.problem_embedding )

    def get_solution_summary_similarity( self, other_snapshot ):
        if not self.solution_embedding or not other_snapshot.solution_embedding:
            raise ValueError( "Both snapshots must have a solution summary embedding to compare." )
        return np.dot( self.solution_embedding, other_snapshot.solution_embedding )

    def get_code_similarity( self, other_snapshot ):
        if not self.code_embedding or not other_snapshot.code_embedding:
            raise ValueError( "Both snapshots must have a code embedding to compare." )
        return np.dot( self.code_embedding, other_snapshot.code_embedding )

    def to_json( self ):
        return json.dumps( self.__dict__ )

    def write_to_file( self ):
        # Generate filename based on first 64 characters of the problem
        filename_base = self.problem[ :64 ]
        # Replace any character that is not alphanumeric or underscore with underscore
        filename_base = "".join( c if c.isalnum() else "-" for c in filename_base )
        # Get the project root directory
        project_root = du.get_project_root()
        # Define the directory where the file will be saved
        directory = f"{project_root}{self.solution_directory}"
        # Get a list of all files that start with the filename base
        existing_files = glob.glob( f"{directory}{filename_base}-*.json" )
        # The count of existing files will be used to make the filename unique
        file_count = len( existing_files )
        # generate the file name
        filename = f"{filename_base}-{file_count}.json"
        # Generate the full file path
        file_path = f"{directory}{filename}"
        # Print the file path for debugging purposes
        print( f"File path: {file_path}" )
        # Write the JSON string to the file
        with open( file_path, "w" ) as f:
            f.write( self.to_json() )

    @classmethod
    def from_json_file( cls, filename ):
        with open( filename, 'r' ) as f:
            data = json.load( f )
        return cls( **data )

today     = SolutionSnapshot( problem="what day is today" )
tommorrow = SolutionSnapshot( problem="what day is tomorrow" )
blah      = SolutionSnapshot( problem="i feel so blah today" )
color     = SolutionSnapshot( problem="what color is the sky" )
date      = SolutionSnapshot( problem="what is today's date" )

today.write_to_file()
# snapshot.to_json()

Elapsed time: 350 milliseconds
Elapsed time: 206 milliseconds
Elapsed time: 341 milliseconds
Elapsed time: 196 milliseconds
Elapsed time: 149 milliseconds
File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/what-day-is-today-1.json


In [35]:
today.write_to_file()

File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/what-day-is-today-0.json.json


In [34]:
snapshots = [ today, tommorrow, blah, color, date ]

for snapshot in snapshots:

    score = today.get_problem_similarity( snapshot )
    print( f"Score: [{score}] for [{snapshot.problem}] == [{today.problem}]" )

Score: [1.0000000721124376] for [what day is today] == [what day is today]
Score: [0.9080752829638264] for [what day is tomorrow] == [what day is today]
Score: [0.7907710414102591] for [i feel so blah today] == [what day is today]
Score: [0.7562339112211474] for [what color is the sky] == [what day is today]
Score: [0.9328160982290687] for [what is today's date] == [what day is today]
